In [ ]:
import pandas as pd
import ast
import math
from collections import Counter
import requests

In [ ]:
def all_same(items):
    if isinstance(items[0], float) and all(math.isnan(x) for x in items):
        return True
    return all(x == items[0] for x in items)

In [ ]:
turk_df = pd.read_excel('NESCent_publications/NESCent_CV.xlsx')

In [ ]:
turk_grouped_df = turk_df.groupby('Input.name').agg(lambda x: x.tolist()[0] if all_same(x.tolist()) else x.tolist())

In [ ]:
stop_urls = ['http://notfound.com', 'http://none', 'http://NA', 'http://na.com', 'http://NA@NA.com', 'http://none.com', 'http://www.NA@NA.com', 'https://NA', 'https://none.com', 'https://notfound.com', 'http://notexactlyfound.com', 'http://notexplicityfound.com', 'http://www.thereisnocv.com', 'file:///C:/Users/user/Downloads/Documents/Ove_Nilsson.pdf', 'httpys://ocs.yale.edu/sites/default/files/files/CV%20to%20ResumeWorkshopfinal.pdf' ]

In [ ]:
all_clean_urls = []

for index, row in turk_grouped_df.iterrows():
    urls = row['Answer.web_url']
    
    # Create array of all urls
    parsed_urls = []
    if isinstance(urls, str):
        if urls[-1] == '|':
            urls = urls[0:-1]
        
        cleaned_url = [x for x in urls.split('|')]
        parsed_urls.extend( cleaned_url)

    elif isinstance(urls, list):
        for url in urls:
            if url[-1] == '|':
                url = url[0:-1]
            parsed_urls.extend(url.split('|'))
        
    # Remove 'https://', 'http://', 'www.' for comparison
    clean_urls = []
    for index, url in enumerate(parsed_urls):
        cleaned_url = url
        if cleaned_url not in stop_urls:
            clean_urls.append(cleaned_url)
    
    all_clean_urls.append(clean_urls)

In [ ]:
url_counters = [Counter(x) for x in all_clean_urls]

In [ ]:
top_5 = [[], [], [], [], []]

for counter in url_counters:
    top_urls = [x[0] for x in counter.most_common()]
    
    for index, url in enumerate(top_urls):
        top_5[index].append(url)
        
    for index in range(len(top_urls), 5):
        top_5[index].append(-1)
        
    

In [ ]:
for index, url_group in enumerate(top_5):
    turk_grouped_df['top_' + str(index+1) + '_url'] =  url_group

In [ ]:
turk_grouped_df = turk_grouped_df.reset_index()

In [ ]:
turk_grouped_df = turk_grouped_df.drop(['Title', 'Description', 'Input.searchlink', 'Answer.web_url'], axis=1)

In [ ]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
from text_scraper import get_text_from_url

In [ ]:
from tqdm import tqdm_notebook as tqdm

raw = [[], [], [], [], []]

with tqdm(total=180) as pbar:
    for i, row in turk_grouped_df.iterrows():
        pbar.update(1)
        for j in range(0,5):
            url = row['top_' + str(j+1) + '_url']

            if url == -1:
                raw[j].append(-1)
            else:
                raw[j].append(get_text_from_url(url, False).replace('\n', " "))
         

In [ ]:
for index, raw_group in enumerate(raw):
    turk_grouped_df['top_' + str(index+1) + '_raw'] =  raw_group

In [ ]:
turk_grouped_df.to_csv('turked_grouped.csv')

In [ ]:
import re

In [ ]:
# Returns True for: (2015) (2013). 2013.
def isCitationYear(test_string):
    if re.findall('^\(\d{4}\)', test_string):
        return True
    
    if re.findall('^\d{4}\.', test_string):
        return True
    
    return False

In [ ]:
isCitationYear('(2015):')

In [ ]:
tolerance = 15

all_publications = []

for i, row in turk_grouped_df.iterrows():
    publications = set()
    
    for j in range(0,5):
        raw = row['top_' + str(j+1) + '_raw']
        
        if raw == -1:
            break
            
        last_name = row['Input.name'].split()[1]

        raw_split = raw.split()
        
        for k, word in enumerate(raw_split):
            if last_name in word and k+1 < len(raw_split):
                current_index = k+1
                current_word = raw_split[current_index]
                found = True
                while(not isCitationYear(current_word)):
                    current_index += 1
                    
                    try:
                        current_word = raw_split[current_index]
                    except IndexError:
                        found = False
                        break
                        
                    if current_index == tolerance:
                        found = False
                        break
                        
                if found and current_index+1 != len(raw_split):
                    title = ''
                    current_index += 1
                    current = raw_split[current_index]
                    while True:
                        title += ' ' + current

                        if '.' in current or '?' in current:
                            break
                        
                        current_index += 1
                    
                        
                        try:
                            current = raw_split[current_index]
                        except:
                            found = False
                            break
                    
                    if found:
                        publications.add(title)           
    all_publications.append(publications)

In [ ]:
all_publications_clean = ['' if x=='set()' else ', '.join(str(y) for y in x) for x in all_publications]

In [ ]:
turk_grouped_df['unconfirmed_publications'] = all_publications_clean

In [ ]:
turk_grouped_df.to_csv('turk_grouped_publications.csv', encoding='utf-8')

In [ ]:
turk_grouped_df.to_excel('turk_grouped_publications.xlsx')

In [ ]:
turk_grouped_df = pd.read_csv('turk_grouped_publications.csv', index_col=0)

In [ ]:
middle_initials_df = pd.read_excel('turk_grouped_publications_with_middle_initial.xlsx')

In [ ]:
turk_grouped_df["Google Scholar Middle Initial"] = middle_initials_df["Google Scholar Middle Initial"]

In [ ]:
turked_grouped_with_middle_initial = turk_grouped_df[turk_grouped_df["Google Scholar Middle Initial"].notnull() ]
turked_grouped_without_middle_initial = turk_grouped_df[turk_grouped_df["Google Scholar Middle Initial"].isnull() ]

In [ ]:
turked_grouped_with_middle_initial.shape

In [ ]:
turked_grouped_without_middle_initial.shape

In [ ]:
turked_grouped_with_middle_initial.to_csv('turk_grouped_with_middle_initial_only.csv')

In [ ]:
turked_grouped_without_middle_initial.to_csv('turk_grouped_without_middle_initial_only.csv')